In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.1.0
Eager execution: True


### Main goal is to predict passenger survival, given characteristics such as gender, age, class, etc. This implementation is based on TF estimator GBT tutorial.

https://www.tensorflow.org/tutorials/estimator/boosted_trees_model_understanding

#### -How to interpret Boosted Trees models both locally and globally.
#### -Local interpretability refers to an understanding of a model’s predictions at the individual example level, while  global interpretability refers to an understanding of the model as a whole. Such techniques can help machine learning (ML) practitioners detect bias and bugs during the model development stage.

#### -For local interpretability, you will learn how to create and visualize per-instance contributions. To distinguish this from feature importances, we refer to these values as directional feature contributions (DFCs).

#### -For global interpretability you will retrieve and visualize gain-based feature importances, permutation feature importances and also show aggregated DFCs.


In [2]:
!pip3 install pandas

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
"Loading the Dataset"

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from IPython.display import clear_output

dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [4]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [5]:
dfeval.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,male,54.0,0,0,51.8625,First,E,Southampton,y
2,female,58.0,0,0,26.5500,First,C,Southampton,y
3,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,male,34.0,0,0,13.0000,Second,D,Southampton,y


#### Preprocessing the data

In [6]:
"Featuring the columns with Categorical Var and Numeric Var"
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
#One-Hot-Encoding
def one_hot_cat_column(feature_name, vocab):
  return fc.indicator_column(
      fc.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(fc.numeric_column(feature_name,
                                           dtype=tf.float32))

### Pipeline

In [7]:
"Create the input functions"
# Samples with the len, such as small datasets.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = (dataset
      .repeat(n_epochs)
      .batch(NUM_EXAMPLES))
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

### Train the model
  You must enable center_bias = True to get DFCs. This will force the model to
  make an initial prediction before using any features (e.g. use the mean of
  the training labels for regression or log odds for classification when
  using cross entropy loss).

In [9]:
params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  'center_bias': True  
}

est = tf.estimator.BoostedTreesClassifier(feature_columns, **params)
# Train model.
est.train(train_input_fn, max_steps=100)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6c3g_pxg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


InternalError: 2 root error(s) found.
  (0) Internal: WhereOp: Could not launch gpuprim::DeviceReduce::Sum to calculate temp_storage_bytes, status: out of memory
	 [[node boosted_trees/transform_features/alone_indicator/to_sparse_input/indices (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/canned/boosted_trees.py:138) ]]
	 [[Func/head/cond/then/_28/input/_123/_319]]
  (1) Internal: WhereOp: Could not launch gpuprim::DeviceReduce::Sum to calculate temp_storage_bytes, status: out of memory
	 [[node boosted_trees/transform_features/alone_indicator/to_sparse_input/indices (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/canned/boosted_trees.py:138) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'boosted_trees/transform_features/alone_indicator/to_sparse_input/indices':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr

In [10]:
# Evaluation.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprected. Please use tf.keras.metrics.AUC instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-27T02:35:43Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6c3g_pxg/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


InternalError: 2 root error(s) found.
  (0) Internal: WhereOp: Could not launch gpuprim::DeviceReduce::Sum to calculate temp_storage_bytes, status: out of memory
	 [[node boosted_trees/transform_features/n_siblings_spouses_indicator/to_sparse_input/indices (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/canned/boosted_trees.py:138) ]]
	 [[boosted_trees/head/assert_range/assert_non_negative/assert_less_equal/Assert/AssertGuard/pivot_f/_13/_324]]
  (1) Internal: WhereOp: Could not launch gpuprim::DeviceReduce::Sum to calculate temp_storage_bytes, status: out of memory
	 [[node boosted_trees/transform_features/n_siblings_spouses_indicator/to_sparse_input/indices (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/canned/boosted_trees.py:138) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'boosted_trees/transform_features/n_siblings_spouses_indicator/to_sparse_input/indices':
  Fi